In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper, CategoricalImputer
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin


Funciones auxiliares

In [2]:
def leer_csv(ruta):
    return pd.read_csv(ruta, dtype={
    'tipodepropiedad':'category', 'ciudad':'category',\
    'provincia':'category'}, parse_dates=[16])

# funcion para setear los superficies
def set_metros(row):
    total = row.metrostotales
    covered = row.metroscubiertos
    if np.isnan(total):
        row.metrostotales = covered
        return row
    if np.isnan(covered):
        row.metroscubiertos = total
        return row
    return row

# Preprocessing 

Imputing Functions

In [3]:
# define numerical imputer
num_imputer = SimpleImputer(strategy='mean')

class SeriesImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        If the Series is of dtype Object, then impute with the most frequent object.
        If the Series is not of dtype Object, then impute with the mean.  

        """
    def fit(self, X, y=None):
        self.fill = X.value_counts().index[0]
        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

# define categorical imputer
cat_imputer = SeriesImputer()

def fill_valores(df, zeros, num_cols, cat_cols):
    df[zeros] = df[zeros].fillna(value=0)
    
    df[num_cols] = num_imputer.fit_transform(df[num_cols])
    for col in cat_cols:
        df[col] = cat_imputer.fit_transform(df[col])
    return df

Encoding functions

In [4]:
def one_hot_encoder(df, columnas):
    for col in columnas:
        df = pd.get_dummies(df, prefix = ['cat'], columns = [col])
    return df

Procesamiento de valores nulos

In [5]:
def processing_nulos(df, eliminar_sin_tipo=False):
    
    # Metros cubiertos y totales
    
    df[['metrostotales', 'metroscubiertos']] = \
    df[['metrostotales', 'metroscubiertos']].apply(set_metros, axis = 1)
    
    # Imputando otras columnas
    
    zeros_cols = ['garages']
    num_cols = ['antiguedad', 'habitaciones', 'banos', 'idzona']
    cat_cols = ['ciudad', 'provincia']
    
    df = fill_valores(df, zeros_cols, num_cols, cat_cols)
    
    return df

Procesamiento de Fechas

In [6]:
def processing_fechas(df, anio, mes, dia):
    df['fecha'] = pd.to_datetime(df['fecha'])
        
    if anio:
        df['anio'] = df['fecha'].dt.year
    if mes:
        df['mes'] = df['fecha'].dt.month
    if dia:
        df['dia'] = df['fecha'].dt.day
    return df

Función suma_columnas

In [7]:
def suma_columnas(df, cols):
    df['suma_'+str(cols)] = sum([df[col] for col in cols])
    return df

_____
### Funcion de preprocessing

In [8]:
def cambiar_valores(x):
    i = 0
    if x.name == 'precioporm2':
        i+=1
        return i
    return x

def encoder_idzona(df_train, df_test): 
    
    df_train['precioporm2'] = df_train['precio'] / df_train['metrostotales']

    precio_m2_por_idzona = df_train.groupby('idzona').agg({'precioporm2' : 'mean'})
    precio_m2_por_idzona.reset_index(inplace = True)
    precio_m2_por_idzona = precio_m2_por_idzona.sort_values(by = 'precioporm2')
    precio_m2_por_idzona

    precio_m2_por_idzona = precio_m2_por_idzona.apply(cambiar_valores)
    precio_m2_por_idzona['precioporm2'] = precio_m2_por_idzona['precioporm2'].cumsum()
    precio_m2_por_idzona

    orden_idzona = precio_m2_por_idzona.set_index('idzona').to_dict()
    orden_idzona = orden_idzona['precioporm2']

    df_train['idzona_ordinal'] = df_train.idzona.map(orden_idzona)
    df_test['idzona_ordinal'] = cat_imputer.fit_transform(df_test.idzona.map(orden_idzona))
    
    return df_train, df_test


In [9]:
def encoder_ciudad(df_train, df_test):
    #filtrando los precios
    df_train_ciudad = df_train.groupby('ciudad').agg({'precio':'mean'}).sort_values('precio').reset_index()
    df_train_ciudad['precio'] = np.log(df_train_ciudad['precio'])
    ciudad_log_precio = df_train_ciudad.set_index('ciudad').to_dict()['precio']
    ciudad_log_precio
    
    df_test['ciudad_ordinal'] = df_test.ciudad.map(ciudad_log_precio)
    df_test['ciudad_ordinal'] = cat_imputer.fit_transform(df_test['ciudad_ordinal'])
    
    df_train['ciudad_ordinal'] = df_train.ciudad.map(ciudad_log_precio)
    
    return df_train, df_test

In [10]:
def preprocessing(guardar_csv=False):
    df_train = leer_csv('train.csv')
    df_test  = leer_csv('test.csv')
    
    ## Fechas
    
    df_train = processing_fechas(df_train, True, True, True)
    df_test = processing_fechas(df_test, True, True, True)
    
    
    ## Encoding de variables categóricas
    
    # Encoding ordinal por precio promedio de 'id zona'
    
    df_train, df_test = encoder_idzona(df_train, df_test)
    
    # One Hot encoding para 'tipo de propiedad'
    df_train = df_train[(df_train['tipodepropiedad'] != 'Hospedaje')&\
                        (df_train['tipodepropiedad'] != 'Garage')]
    df_train['tipodepropiedad'].cat.remove_unused_categories()
    df_train = one_hot_encoder(df_train, ['tipodepropiedad'])
    df_test = one_hot_encoder(df_test, ['tipodepropiedad'])
    
    # Encoding ordinal de las ciudades
    
    df_train, df_test = encoder_ciudad(df_train, df_test)
    
    ## Otras Features
    
    # 'cant_extras' = 'gimnasio'+'usosmultiples'+'piscina'
    extras = ['gimnasio', 'usosmultiples', 'piscina']
    df_train = suma_columnas(df_train, extras)
    df_test = suma_columnas(df_test, extras)
    
    # 'cant_cercanos' = 'escuelascercanas'+'centroscomercialescercanos'
    cercanos = ['escuelascercanas', 'centroscomercialescercanos']
    df_train = suma_columnas(df_train, cercanos)
    df_test = suma_columnas(df_test, cercanos)
    
    ## Nulos
    
    df_train = processing_nulos(df_train, eliminar_sin_tipo=True)
    df_test = processing_nulos(df_test)
    
    ## Eliminamos columnas no utilizadas
    df_train.drop(columns=['fecha', 'id', 'idzona', 'titulo', 'descripcion','direccion',\
                           'lat', 'lng', 'provincia','ciudad','gimnasio','usosmultiples',\
                           'piscina','escuelascercanas','centroscomercialescercanos',\
                           'metroscubiertos','precioporm2', 'cat_Garage', 'cat_Hospedaje'], inplace=True)
    df_test.drop(columns=['fecha', 'id', 'idzona', 'titulo', 'descripcion', 'direccion',\
                          'lat', 'lng','provincia', 'ciudad','gimnasio','usosmultiples',\
                          'piscina','escuelascercanas','centroscomercialescercanos',\
                          'metroscubiertos'], inplace=True)
    
    ## Agregamos columna con log(precio)
    df_train['log_precio'] = np.log(df_train['precio'])
    
    if guardar_csv:
        df_train.to_csv('train_preproc.csv')
        df_test.to_csv('test_preproc.csv')
    return df_train, df_test

In [11]:
df_train, df_test = preprocessing(guardar_csv=False)

In [12]:
# tiene 2 columnas más: precio y log_precio
df_train.columns.nunique()

36

In [13]:
df_test.columns.nunique()

34